# Simplexn

In [ ]:
using BenchmarkTools
using SparseArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

## simplexFacets(simplices::Cells)::Cells
Genera il `(d-1)`-scheletro (insieme delle `faccette`) di un `d`-complesso simpliciale.

Ogni faccetta è generata come differenza tra uno dei `d`-simplessi di `simplices` e uno dei suoi vertici `v` tramite la funzione `setdiff`. In questo modo si ottiene una `facet` di dimensione `d-1` rispetto al simplesso da cui è generata.

Ogni faccetta è poi aggiunta a un array (`push!`) che viene restituito come output dopo essere stato ordinato.

In [ ]:
function simplexFacets(simplices)
	out = Array{Int64,1}[]
	for simplex in simplices
		for v in simplex
			facet = setdiff(simplex,v)
			push!(out, facet)
		end
	end
	# remove duplicate facets
	return sort(collect(Set(out)))
end

In [ ]:
@btime simplexFacets([[1, 2, 3, 5],[2, 3, 5, 6],[3, 5, 6, 7],[2, 3, 4, 6],[3, 4, 6, 7],[4, 6, 7, 8]])

## simplex(n::Int, fullmodel=false::Bool)::Union{Lar.LAR, Lar.LARmodel}
Genera il modello `LAR` di un simplesso `n`-dimensionale in un `n`-spazio.

Restituisce `V`, la matrice dei vertici, e `CV`, la cella che rappresenta il simplesso. Se il parametro `fullmodel==true`, allora `CV` viene arricchito con tutte le facce del simplesso di dimensione da `1` a `n` con l'utilizzo della funzione `simplexFacets`.

In [ ]:
function simplex(n, fullmodel=false)
	eye(n) = LinearAlgebra.Matrix{Int}(I,n,n)
	V = [zeros(n,1) eye(n)]
	CV = [collect(1:n+1)]
	if fullmodel == false
		return V,CV
	else
		h = n
		cells = [CV]
		while h != 0
			push!(cells, simplexFacets(cells[end]))
			h -= 1
		end
		return V,reverse(cells)
	end
end

In [ ]:
@btime simplex(3, true)

## extrudeSimplicial(model::LAR, pattern::Array)::LAR
Algoritmo per l'estrusione di un complesso simpliciale. Può essere applicato a modelli `0`-, `1`-, `2`-, ... dimensionali per ottenere modelli di dimensione superiore (`1`-, `2`-, `3`-, ...).



In [ ]:
function extrudeSimplicial(model::Lar.LAR, pattern)
    V = [model[1][:,k] for k=1:size(model[1],2)]
    FV = model[2]
    d, m = length(FV[1]), length(pattern)
    coords = collect(cumsum(append!([0], abs.(pattern))))
    offset, outcells, rangelimit, i = length(V), [], d*m, 0
    for cell in FV
        i += 1
        tube = [v+k*offset for k in range(0, length=m+1) for v in cell]
        cellTube = [tube[k:k+d] for k in range(1, length=rangelimit)]
        if i==1 outcells = reshape(cellTube, d, m)
        else outcells = vcat(outcells, reshape(cellTube, d, m)) end
    end
    cellGroups = []
    for i in 1:size(outcells, 2)
        if pattern[i]>0
            cellGroups = vcat(cellGroups, outcells[:, i])
        end
    end
    outVertices = [vcat(v, [z]) for z in coords for v in V]
    cellGroups = convert(Array{Array{Int, 1}, 1}, cellGroups)
    outModel = outVertices, cellGroups
    hcat(outVertices...), cellGroups
end
function extrudeSimplicial(model::Union{Any,Lar.Cells}, pattern)
    V,FV = model
    d, m = length(FV[1]), length(pattern)
    coords = collect(cumsum(append!([0], abs.(pattern))))
    offset, outcells, rangelimit, i = length(V), [], d*m, 0
    for cell in FV
        i += 1
        tube = [v+k*offset for k in range(0, length=m+1) for v in cell]
        cellTube = [tube[k:k+d] for k in range(1, length=rangelimit)]
        if i==1 outcells = reshape(cellTube, d, m)
        else outcells = vcat(outcells, reshape(cellTube, d, m)) end
    end
    cellGroups = []
    for i in 1:size(outcells, 2)
        if pattern[i]>0
            cellGroups = vcat(cellGroups, outcells[:, i])
        end
    end
    outVertices = [vcat(v, [z]) for z in coords for v in V]
    cellGroups = convert(Array{Array{Int, 1}, 1}, cellGroups)
    outModel = outVertices, cellGroups
    hcat(outVertices...), cellGroups
end

In [ ]:
pattern = repeat([1,2,-3],outer=4)
@btime extrudeSimplicial(([[0,0] [1,0] [2,0] [0,1] [1,1] [2,1] [0,2] [1,2] [2,2]], [[1,2,4],[2,3,5],[3,5,6],[4,5,7],[5,7,8],[6,8,9]]), pattern)

## simplexGrid(shape::Array)::LAR
Genera una decomposizione simpliciale di una griglia cubica formata da `d`-cuboidi, dove `d` è la lunghezza dell'array `shape`, parametro in input che definisce la forma della griglia.

Facendo uso di `extrudeSimplicial` si ottiene un modello LAR del complesso simpliciale formato da vertici `V` e celle `CV`.

In [ ]:
function simplexGrid(shape)
    model = [[]], [[1]]
    for item in shape
        model = extrudeSimplicial(model, fill(1, item))
    end
    V, CV = model
    V = convert(Array{Float64,2}, V)
    return V, CV
end

In [ ]:
@btime simplexGrid([1,1,1])